# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234378282640                   -0.50    7.0         
  2   -7.249799331626       -1.81       -1.39    1.0   10.5ms
  3   -7.250847701358       -2.98       -1.81    2.0   13.1ms
  4   -7.250993969754       -3.83       -1.89    2.0   12.3ms
  5   -7.251326988711       -3.48       -2.60    1.0   9.28ms
  6   -7.251337877176       -4.96       -3.08    2.0   13.3ms
  7   -7.251338681425       -6.09       -3.47    3.0   12.6ms
  8   -7.251338776230       -7.02       -3.90    1.0   10.9ms
  9   -7.251338797509       -7.67       -4.77    1.0   11.2ms
 10   -7.251338798675       -8.93       -5.21    3.0   13.7ms
 11   -7.251338798697      -10.65       -5.61    2.0   11.7ms
 12   -7.251338798704      -11.21       -6.21    1.0   73.2ms
 13   -7.251338798704      -12.12       -6.48    3.0   12.4ms
 14   -7.251338798705      -12.99       -6.93    1.0   8.91ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04083287112995299
[ Info: Arnoldi iteration step 2: normres = 0.8398331148359578
[ Info: Arnoldi iteration step 3: normres = 0.5085684707923668
[ Info: Arnoldi iteration step 4: normres = 0.3862912572205094
[ Info: Arnoldi iteration step 5: normres = 0.25068144902205247
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.04e-02, 7.56e-02, 1.79e-01, 1.58e-01, 1.03e-02)
[ Info: Arnoldi iteration step 6: normres = 0.6203999219341219
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.47e-03, 1.08e-01, 5.78e-01, 1.41e-01, 1.34e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08584513955617142
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.61e-04, 4.47e-02, 2.66e-02, 3.67e-02, 4.44e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13166599353285327
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.48e-